In [1]:

import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-189ca005-d852-4370-b4dd-50a8499acec6',
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token',
    'api_key': 'mAo40B3h-s0GPQnsakluqLLmGPlQkbNmHif2xzcvDY6C'
}

configuration_name = 'os_9bed24a8267a4d0ea5db2140346dee9d_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df_data_1 = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('ASA_log.csv', 'capstoneproject-donotdelete-pr-68gfgxkh2rslzx'))
df_data_1.take(5)


Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20190916200805-0000
KERNEL_ID = 2e511fdf-e281-4554-a8aa-9781373c3f23


[Row(sourcePort='52217', destinationPort='2000', protocolName='tcp_ip', IPgeo=None, deviceId='31410', event_category='5010', categoryDescription='Misc Exploit', eventDescription='Unrecognized Palo Alto PA Series Vulnerability Exploit Threat Event', relevance='8', credibility='10', severity='9', magnitude='9', Event_DateTime='Mar 14, 2019, 8:26:53 PM', eventCount='1'),
 Row(sourcePort='51405', destinationPort='80', protocolName='tcp_ip', IPgeo=None, deviceId='31410', event_category='7024', categoryDescription='Information Leak', eventDescription='A directory traversal vulnerability has been discovered in parsing malformed HTTP requests. This vulnerability is due to a lack of proper checks in HTTP URI requests. A successful attack could result in access to sensitive information that could further aid in other attacks.', relevance='8', credibility='10', severity='1', magnitude='6', Event_DateTime='Mar 14, 2019, 8:26:53 PM', eventCount='1'),
 Row(sourcePort='36002', destinationPort='445', 

In [3]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.sql.functions import unix_timestamp, from_unixtime, to_timestamp
from sklearn.model_selection import train_test_split
import numpy as np
# Use spark.sql to create a pyspark dataframe
# cast data from stringtype to integertype where appropriate
# data is either an integer or string

df_data_1.createOrReplaceTempView("firewall")
data = spark.sql('SELECT * FROM firewall')
data = data.withColumn('sourcePort', data['sourcePort'].cast(IntegerType()))
data = data.withColumn('destinationPort', data['destinationPort'].cast(IntegerType()))
data = data.withColumn('deviceId', data['deviceId'].cast(IntegerType()))
data = data.withColumn('event_category', data['event_category'].cast(IntegerType()))
data = data.withColumn('relevance', data['relevance'].cast(IntegerType()))
data = data.withColumn('credibility', data['credibility'].cast(IntegerType()))
data = data.withColumn('severity', data['severity'].cast(IntegerType()))
data = data.withColumn('magnitude', data['magnitude'].cast(IntegerType()))
data = data.withColumn('eventCount', data['eventCount'].cast(IntegerType()))
data = data.drop('Event_DateTime')
data = data.fillna('Unknown')
data.show()


+----------+---------------+------------+-------------+--------+--------------+-------------------+--------------------+---------+-----------+--------+---------+----------+
|sourcePort|destinationPort|protocolName|        IPgeo|deviceId|event_category|categoryDescription|    eventDescription|relevance|credibility|severity|magnitude|eventCount|
+----------+---------------+------------+-------------+--------+--------------+-------------------+--------------------+---------+-----------+--------+---------+----------+
|     52217|           2000|      tcp_ip|      Unknown|   31410|          5010|       Misc Exploit|Unrecognized Palo...|        8|         10|       9|        9|         1|
|     51405|             80|      tcp_ip|      Unknown|   31410|          7024|   Information Leak|A directory trave...|        8|         10|       1|        6|         1|
|     36002|            445|      tcp_ip|      Unknown|   31410|          7024|   Information Leak|This alert indica...|        8|     

In [6]:
# here we change string columns to indexed columns with string indexer
# then we onehotencode those columns and use a vector assembler to create a features column for them
categorical_columns = ['protocolName', 'IPgeo', 'event_category','eventDescription', 'categoryDescription']
output_columns = ['pnlbl', 'ipglbl', 'evcatlbl', 'ecdlbl', 'catdeslbl']
indexers = [
    StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c), handleInvalid="keep")
    for c in categorical_columns
]

encoders = [OneHotEncoder(dropLast=False, inputCol=indexer.getOutputCol(), outputCol="{0}_encoded".format(indexer.getOutputCol()))
           for indexer in indexers
          ]
assembler = VectorAssembler(inputCols=[encoder.getOutputCol() for encoder in encoders], outputCol="features")
pipeline = Pipeline(stages=indexers + encoders+[assembler])
model=pipeline.fit(data)
transformed = model.transform(data)
dataindex = transformed
# drop the original columns before being transformed, I'm unsure if this is correct but
# the original column types can't be passed into keras
for col in categorical_columns:
    dataindex = dataindex.drop(col)
x = dataindex.drop('features')
y = dataindex[['features']]


In [7]:
# try to change the datafram to a numpy array so it can be fed into keras
x_np = np.array(x.select('sourcePort',
 'destinationPort',
 'deviceId',
 'relevance',
 'credibility',
 'severity',
 'magnitude',
 'eventCount',
 'protocolName_indexed',
 'IPgeo_indexed',
 'event_category_indexed',
 'protocolName_indexed_encoded',
 'IPgeo_indexed_encoded',
 'event_category_indexed_encoded',).collect())
# wont acept a sequence as an array value

ValueError: setting an array element with a sequence.

In [ ]:
# this was the setup for the neural net but as I haven't been able to pass any of the data into it in
# the correct format I haven't updated it yet
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import RMSprop, SGD

model = Sequential()

#get number of columns in training data
notnneded, n_cols_2 =xtrain_np.shape 

#add layers to model
model.add(Dense(250, activation='relu', input_shape=(n_cols_2,)))
model.add(Dropout(0.5))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(xtrainnp, ytrainnp, epochs=10, batch_size=64)
score = model.evaluate(xtestnp, ytestnp, batch_size=128)